In [1]:
#Importamos librerías
import pandas as pd

In [2]:
#Archivo donde tomamos el set de datos correspondiente a subte para su limpieza y normalización
#https://cdn.buenosaires.gob.ar/datosabiertos/datasets/sbase/subte-viajes-molinetes/molinetes-2020.zip
input_file_name = 'subte.csv'

In [3]:
#Definimos el archivo donde guardar el set de datos luego de la limpieza y normalización
output_file_name = "subte.xlsx"
output_sheet_name = "subte"

In [4]:
#Carga del dataframe
df_subte = pd.read_csv(input_file_name)

In [5]:
#Previsualización de los primeros registros
df_subte.head()

,FECHA,DESDE,HASTA,LINEA,MOLINETE,ESTACION,pax_pagos,pax_pases_pagos,pax_franq,pax_TOTAL
0,01/01/2020,08:00:00,08:15:00,LineaA,LineaA_Acoyte_N_Turn01,Acoyte,1.0,0.0,0.0,1.0
1,01/01/2020,08:00:00,08:15:00,LineaA,LineaA_Carabobo_E_Turn02,Carabobo,6.0,0.0,0.0,6.0
2,01/01/2020,08:00:00,08:15:00,LineaA,LineaA_CBarros_N_Turn03,Castro Barros,3.0,0.0,1.0,4.0
3,01/01/2020,08:00:00,08:15:00,LineaA,LineaA_CBarros_S_Turn02,Castro Barros,2.0,0.0,0.0,2.0
4,01/01/2020,08:00:00,08:15:00,LineaA,LineaA_Congreso_N_Turn03,Congreso,2.0,0.0,0.0,2.0


In [6]:
#Cantidad de filas y columnas
df_subte.shape

(5781006, 10)

In [7]:
#Filtrado de registros elinando nulos
df_subte = df_subte[df_subte['FECHA'].notna()]

In [8]:
#Imputación del año según fecha de origen
df_subte["Año"] = pd.DatetimeIndex(df_subte['FECHA']).year

In [9]:
#Imputación del mes según fecha de origen
df_subte["Mes"] = pd.DatetimeIndex(df_subte['FECHA']).month

In [10]:
#Imputación del tipo de registro
df_subte['Tipo'] = 'Subte'

In [11]:
#Agrupación de datos
df = df_subte.groupby(['Año','Mes','Tipo'],as_index=False).sum()[['Año','Mes','Tipo','pax_TOTAL']].rename({'pax_TOTAL':'Total'},axis=1)
df

,Año,Mes,Tipo,Total
0,2020,1,Subte,14431260.0
1,2020,2,Subte,14379523.0
2,2020,3,Subte,16665570.0
3,2020,4,Subte,2532868.0
4,2020,5,Subte,2769479.0
5,2020,6,Subte,3566230.0
6,2020,7,Subte,3417198.0
7,2020,8,Subte,3122951.0
8,2020,9,Subte,3629713.0
9,2020,10,Subte,4966696.0


In [12]:
#Definición de intervalo según distribución de datos
df['Total'] = df['Total'].astype(int)
df['Intervalo']=pd.cut(df['Total'],10,labels=[1,2,3,4,5,6,7,8,9,10])
df

,Año,Mes,Tipo,Total,Intervalo
0,2020,1,Subte,14431260,9
1,2020,2,Subte,14379523,9
2,2020,3,Subte,16665570,10
3,2020,4,Subte,2532868,1
4,2020,5,Subte,2769479,1
5,2020,6,Subte,3566230,1
6,2020,7,Subte,3417198,1
7,2020,8,Subte,3122951,1
8,2020,9,Subte,3629713,1
9,2020,10,Subte,4966696,2


In [13]:
#Definición relación de los datos entre diciembre y enero
enero = df.loc[ df.Mes == 1, ['Intervalo']].iloc[0,0]
diciembre = df.loc[ df.Mes == 12, ['Intervalo']].iloc[0,0]
diciembre_enero = int(((diciembre - enero)/enero)*100)
if diciembre_enero > 0:
    diciembre_enero = '+' + str(diciembre_enero) + ' %'
else:
    diciembre_enero = str(diciembre_enero) + ' %'

In [14]:
#Imputación relación de los datos entre diciembre y enero
df['Diciembre Vs Enero'] = diciembre_enero
df

,Año,Mes,Tipo,Total,Intervalo,Diciembre Vs Enero
0,2020,1,Subte,14431260,9,-77 %
1,2020,2,Subte,14379523,9,-77 %
2,2020,3,Subte,16665570,10,-77 %
3,2020,4,Subte,2532868,1,-77 %
4,2020,5,Subte,2769479,1,-77 %
5,2020,6,Subte,3566230,1,-77 %
6,2020,7,Subte,3417198,1,-77 %
7,2020,8,Subte,3122951,1,-77 %
8,2020,9,Subte,3629713,1,-77 %
9,2020,10,Subte,4966696,2,-77 %


In [15]:
#Resguardo de los datos para su posterior análisis 
writer = pd.ExcelWriter(output_file_name, engine='xlsxwriter')

df.to_excel(writer, index=False, sheet_name=output_sheet_name)

workbook  = writer.book
worksheet = writer.sheets[output_sheet_name]

format = workbook.add_format()
format.set_align('center')
format.set_align('vcenter')

worksheet.set_column('A:F', 30, format)

writer.save()